In [1]:
import gmsh
import sys
import numpy as np
from typing import List, Tuple

In [2]:
def get_coord(R, rho, psi, phi):
    '''
    Дает декартову координату точки по её координатам на торе
    '''
    x = (R + rho * np.cos(psi)) * np.cos(phi)
    y = (R + rho * np.cos(psi)) * np.sin(phi)
    z = rho * np.sin(psi)
    return x, y, z

In [3]:
def get_dots(R, rho, N = 10, lc=1e-1):
    '''
    Дает все точки тора
    '''
    
    dots = []
    for psi in np.linspace(0, 2*3.14, N):
        line = []
        for phi in np.linspace(0, 2*3.14, N):
            x, y, z = get_coord(R, rho, psi, phi)
            line.append(gmsh.model.geo.addPoint(x, y, z, lc))
        dots.append(line)
    return dots

In [4]:
def make_thor(R, rho, N = 10, lc=1e-1):
    '''
    Делает тор
    '''
    dots = get_dots(R, rho, N, lc)
    
    planes = []
    
    for i in range(len(dots)):
        for j in range(len(dots[0])):
            lines = []
            lines.append(gmsh.model.geo.addLine(dots[i       % N][j       % N], dots[i       % N][(j + 1) % N]))
            lines.append(gmsh.model.geo.addLine(dots[i       % N][(j + 1) % N], dots[(i + 1) % N][(j + 1) % N]))
            lines.append(gmsh.model.geo.addLine(dots[(i + 1) % N][(j + 1) % N], dots[(i + 1) % N][j       % N]))
            lines.append(gmsh.model.geo.addLine(dots[(i + 1) % N][j       % N], dots[i       % N][j       % N]))
            
            planes.append(gmsh.model.geo.addPlaneSurface([gmsh.model.geo.addCurveLoop(lines)]))
    
    thor = gmsh.model.geo.addSurfaceLoop(planes)
    gmsh.model.geo.addVolume([thor])

In [10]:
gmsh.initialize()
gmsh.model.add("t1")

make_thor(2, 1, 10)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("t1.msh")

if '-nopopup' not in sys.argv:
    gmsh.fltk.run()
    
gmsh.finalize()

OSError: exception: access violation writing 0x0000000000000CE9